In [5]:
import numpy as np
import pandas as pd

In [6]:
# import data
df = pd.read_pickle('analysis.p')
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [9]:
def getcount(decision, direction):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [5]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [lnin, expin, strexpin, plwcin]
outcounts = [lnout, expout, strexpout, plwcout]
unkcounts = [lnunk, expunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts

[3, 4, 5, 40]
[2, 101, 76, 0]
[181, 72, 107, 148]


In [8]:
df.iloc[0:3]

,Domain,Subdomain,num_edges,Graph_order,Weighted,Directed,Bipartite,Multigraph,Multiplex,Component,...,n,alpha,xmin,ntail,Lpl,ppl,dexp,dln,dstrexp,dplwc
Small_plant-pollinator_web_Small_plant-pollinator_web_Biological_Food_web_n2.gml_bipartitefull_weightedsimplifieddistribution.txt,Biological,Food_web,141,2,simplified,0,full,0,0,connected,...,47,2.46,4,32,-82.4076,0.121,,,,
Freshwater_stream_webs_SuttonSp_Biological_Food_web_n3.gml_directedindistribution.txt,Biological,Food_web,391,3,0,in,0,0,0,entire,...,79,1.53,4,22,-94.0899,0,,,,
Freshwater_stream_webs_SuttonSp_Biological_Food_web_n3.gml_directedoutdistribution.txt,Biological,Food_web,391,3,0,out,0,0,0,entire,...,79,5.5,13,13,-29.2702,0.937,,,,
